In [56]:
import geopandas as gpd
import turfpy
from geojson import Point, Feature, FeatureCollection
from ipyleaflet import Map, GeoJSON, LayersControl, Marker
import folium
import numpy as np
import json
from turfpy.measurement import nearest_point, distance

In [2]:
geojson_path = "E:/Satyukt/paired_pilot/data/shp/VILLAGE.shp"

gdf = gpd.read_file(geojson_path)

## Calculate the Centroid
gdf['centroid'] = [x.centroid for x in gdf.geometry]

#For meters
gdf = gdf.to_crs(3763)

C:\Users\Asus\miniconda3\envs\gis\lib\site-packages\pandas\core\dtypes\cast.py:128: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [3]:
## Initialize the Map
m = Map(center = [18.42,77.35], zoom = 12)

# Convert the centroids in gdf to a FeatureCollection
feature_collection = FeatureCollection([Feature(geometry=Point(list(gdf['centroid'][i].coords)[0])) for i in range(len(gdf))])

In [88]:
def getNearestPointDistance(pt,dist):
    '''
    Returns the Nearest Point in the Feature collection
    It takes two arguments:
        pt >> shapely.coordinates.sequence
        dist >> Its the boolean value which if set to true, 
        Returns the distance in meters to that point
    '''
    global feature_collection
    point = Point(pt)
    if dist == True:
        return (distance(point,nearest_point(point, feature_collection)))
    
    return nearest_point(point, feature_collection)


def bufferPoint(pt):
    '''
    Takes the Shapely.geometry.point object and returns the buffer of 50 meters
    '''
    return gpd.GeoSeries(pt).buffer(50)


def pointsInBuffer(buff):
    '''
    Takes the Buffer of a point and returns all the points which lies inside of the Polygon
    It returns the GeoDataFrame of the Points.
    '''
    global gdf
    return gdf.iloc[[buffer.contains(gdf['centroid'][i])[0] for i in range(len(gdf))]] 

---
Turf

In [89]:
pointsInBuffer(bufferPoint(gdf['centroid'][0]))

,KARNA_ID,NAME_11,SUB_DIST11,DISTRICT11,STATE_UT,CENSUS2011,geometry,centroid
0,1,Mukhed,Aurad,Bidar,Karnataka,2955805456599970,"POLYGON ((11338600.812 4139446.564, 11332106.8...",POINT (77.28518620189236 18.461010115602893)
1,2,Bawalgaon,Aurad,Bidar,Karnataka,2955805456599972,"POLYGON ((11353473.795 4157547.603, 11353574.2...",POINT (77.3589626888698 18.42177201032195)
2,3,Nandi Bijalgaon,Aurad,Bidar,Karnataka,2955805456599971,"POLYGON ((11347868.760 4140358.020, 11347566.0...",POINT (77.30227896672584 18.42363933857228)
3,4,Chikli Udgir,Aurad,Bidar,Karnataka,2955805456599969,"POLYGON ((11347868.760 4140358.020, 11348002.2...",POINT (77.26165389523375 18.40081223598321)
4,5,Hokrana,Aurad,Bidar,Karnataka,2955805456599973,"POLYGON ((11352425.007 4154583.083, 11352441.6...",POINT (77.33008207087087 18.400810467095308)
...,...,...,...,...,...,...,...,...
29832,28498,Bandipura,Gundlupet,Chamarajanagar,Karnataka,2957805576619597,"POLYGON ((13828012.583 3093285.004, 13828870.4...",POINT (76.63714543231514 11.671778910880079)
29833,28499,Mangala,Gundlupet,Chamarajanagar,Karnataka,2957805576619596,"POLYGON ((13834967.695 3097878.631, 13835196.8...",POINT (76.66269469275353 11.659348771566169)
29834,28500,Jakkahalli,Gundlupet,Chamarajanagar,Karnataka,2957805576619595,"POLYGON ((13845276.364 3111669.016, 13845214.2...",POINT (76.68457472800331 11.647394644650959)
29835,28501,Yelachatti,Gundlupet,Chamarajanagar,Karnataka,2957805576619599,"POLYGON ((13856695.403 3113681.094, 13855237.9...",POINT (76.70935877817683 11.649084587165197)
